In [6]:
import numpy as np
from matplotlib.pyplot import *
from scipy.sparse import diags  # Pour les matrices diagonales

# REMARQUE : éviter d'appeler une variable ou une fonction "lambda" car déjà prédéfini par python

# CE QUI SUIT EST UNE BASE POUR IMPLEMENTER LES FONCTIONS DES DEUX METHODES

# La fonction suivante permet de discrétiser un intervalle [0,1] en N+1 noeuds équidistants
# (c'est-à-dire on "tranche" notre colonne de terre en N+1 morceaux)

def intervalle_discret(profondeur,N): # Pour commencer on peut prendre par exemple N=10
    x=np.linspace(0,profondeur,N+1)
    return x

# La fonction suivante permet de calculer le delta_x à partir de l'intervalle discrétisé par la fonction ci-dessus
def calcul_delta_x(intervalle_discret):
    return intervalle_discret[1]-intervalle_discret[0]

# La fonction suivante permet de calculer la constante alpha de l'équation différentielle 
# On détermine alpha grâce aux valeurs de la conductivité thermique du sol et à sa capacité thermique volumique
# ATTENTION: il faut bien utiliser les unités SI

def calcul_alpha(lambda_conductivite_thermique_sol,capacite_thermique_volumique_sol):
    alpha=lambda_conductivite_thermique_sol/capacite_thermique_volumique_sol
    return alpha

# La fonction suivante permet de calculer la valeur de lambda qui servira dans le calcul des températures de la colonne au cours du temps

def calcul_lambda(alpha,delta_x,delta_t):
    lamb=alpha*delta_t/((delta_x)**2)
    return lamb

# La fonction suivante permet d'initialiser les températures de notre colonne de terre à l'instant t=0 dans un vecteur colonne de dimensions N-1 (et pas N attention)
# Il s'agit d'un choix arbitraire étant donné que les valeurs vont toujours finir par converger vers les températures à l'équilibre thermodynamique de la colonne de terre 
# On choisit ici d'initialiser notre colonne avec un température uniforme qui vaut 0
# ATTENTION: dans le programme qui étudiera la température sur plusieurs heures, il ne faudra pas réinitialiser à 0 à chaque nouvelle heure mais seulement au début de la première
def initialisation_zeros(N):
    T=np.zeros((N-1,1)) # création d'un vecteur colonne avec que des zéros
    return T
    
# La fonction suivante permet de créer le vecteur qui sera utilisé pour calculer le vecteur T à l'étape k+1
# Il n'y a pas vraiment de nom pertinent pour cette fonction
# extremite fait référence aux températures aux extrémités de la colonne de terre (car elles interviennent uniquement dans ce vecteur)

def extremite(N,lamb,Tg,Td):
    ext=np.zeros((N-1,1))
    ext[0]=lamb*Tg
    ext[-1]=lamb*Td
    
    return ext


# On passe maintenant aux fonctions spécifiques à chaque méthode
# "e" correspond à explicite et "i" à implicite 

# METHODE EXPLICITE

# La fonction suivante permet de créer la matrice diagonale de taille (N-1)*(N-1)
# Cette matrice sera utilisée pour calculer le vecteur T à l'étape k+1 avec la méthode explicite
# ATENTION : la matrice diagonale est différente pour la méthode implicite

def matrice_diag_e(N,lamb):
    k = np.array([lamb*np.ones(N-2),1-2*lamb*np.ones(N-1),lamb*np.ones(N-2)])
    offset = [-1,0,1]
    A = diags(k,offset).toarray()
    return A

# La fonction suivante est l'implémentation de la méthode explicite
# Cette fonction reprend d'autres fonctions vues plus haut
# Elle retourne par ailleurs le vecteur colonne T au bout d'un certain temps donné par n*delta_t
# Chaque étape (soit chaque boucle) correspond à un avancement de delta_t dans le temps
# Plus on fait de boucles, plus on se rappoche de la valeur théorique des températures à l'équilibre thermodynamique dans la colonne
# En effet, on atteint à un certain moment des températures constantes dans la colonne de terre qar on a atteint l'équilibre
# Pour rappel on considère que l'on s'intéresse à une colonne de terre dont la température est initialement nulle et dont on applique une certaine température au niveau du sol et au niveau de la dernière couche de sol)
# Ainsi, pour plus de logique, on suppose que la température imposée sur la dernière couche du sol est égale à la température initiale de la colonne
# (cela semblerait étrange d'imposer une température dans le sol à la colonne de terre)


def methode_explicite(profondeur,N,Tg,Td,T_initial,lambda_conductivite_thermique_sol,capacite_thermique_volumique_sol,delta_t,n):
    
    x=intervalle_discret(profondeur,N)
    delta_x=calcul_delta_x(x)
    alpha=calcul_alpha(lambda_conductivite_thermique_sol,capacite_thermique_volumique_sol)
    lamb=calcul_lambda(alpha,delta_x,delta_t)
    
    ext=extremite(N,lamb,Tg,Td)
    
    A=matrice_diag_e(N,lamb)
    T=T_initial
    
    for i in range(1,n+1):
        T=A.dot(T)+ext
        
    return T


# METHODE IMPLICITE

# La fonction suivante permet de créer la matrice diagonale de taille (N-1)*(N-1)
# Cette matrice sera utilisée pour calculer le vecteur T à l'étape k+1 avec la méthode implicite
# ATENTION : cette matrice diagonale est différente pour la méthode explicite

def matrice_diag_i(N,lamb):
    k = np.array([-lamb*np.ones(N-2),1+2*lamb*np.ones(N-1),-lamb*np.ones(N-2)])
    offset = [-1,0,1]
    A = diags(k,offset).toarray()
    return A

# La fonction suivante est l'implémentation de la méthode explicite
# Cette fonction reprend d'autres fonctions vues plus haut
# Elle retourne par ailleurs le vecteur colonne T au bout d'un certain temps donné par n*delta_t
# [Explications cf méthode explicite]

def methode_implicite(profondeur,N,Tg,Td,T_initial,lambda_conductivite_thermique_sol,capacite_thermique_volumique_sol,delta_t,n):
    
    x=intervalle_discret(profondeur,N)
    delta_x=calcul_delta_x(x)
    alpha=calcul_alpha(lambda_conductivite_thermique_sol,capacite_thermique_volumique_sol)
    lamb=calcul_lambda(alpha,delta_x,delta_t)
    
    ext=extremite(N,lamb,Tg,Td)
    
    A=matrice_diag_i(N,lamb)
    T=T_initial
    
    for i in range(1,n+1):
        T= np.linalg.solve(A,T+ext)
    return T




In [7]:
profondeur=1
N=10

Tg=10 
Td=0 
T_initial=initialisation_zeros(N) #création vecteur nul de taille N


lambda_conductivite_thermique_sol=2.1
capacite_thermique_volumique_sol=1674*10**3

#delta_x=0.1   
delta_t=10    

n=100000


In [8]:
# Méthode explicite

T_e=methode_explicite(profondeur,N,Tg,Td,T_initial,lambda_conductivite_thermique_sol,capacite_thermique_volumique_sol,delta_t,n)

# Affichage des températures de la colonne de terre après un temps de n*delta_t

print('T explicite =',T_e)


T explicite = [[8.99999094]
 [7.99998277]
 [6.99997629]
 [5.99997213]
 [4.99997069]
 [3.99997213]
 [2.99997629]
 [1.99998277]
 [0.99999094]]


In [9]:
# Méthode implicite

T_i=methode_implicite(profondeur,N,Tg,Td,T_initial,lambda_conductivite_thermique_sol,capacite_thermique_volumique_sol,delta_t,n)

# Affichage des températures de la colonne de terre après un temps de n*delta_t

print('T implicite =',T_i)

T implicite = [[8.99999093]
 [7.99998275]
 [6.99997626]
 [5.99997209]
 [4.99997065]
 [3.99997209]
 [2.99997626]
 [1.99998275]
 [0.99999093]]


In [10]:
# Méthode explicite

def evolution_mois_e(liste_t,profondeur,N,Td,lambda_conductivite_thermique_sol,capacite_thermique_volumique_sol):
    taille=len(liste_t)
    
    T_initial=initialisation_zeros(N)
    delta_t=1
    n=100000
    
    T_courbes=[]
    
    for i in range(taille):
        Tg=liste_t[i]
        T_e=methode_explicite(profondeur,N,Tg,Td,T_initial,lambda_conductivite_thermique_sol,capacite_thermique_volumique_sol,delta_t,n)
        T_initial=T_e
        T_courbes.append(T_e)
        
    return T_courbes


def evolution_annee_e(liste_t,profondeur,N,Td,lambda_conductivite_thermique_sol,capacite_thermique_volumique_sol):
    taille=len(liste_t)
    
    T_initial=initialisation_zeros(N)
    delta_t=1
    n=100000
    
    T_courbes=[]
    
    for i in range(taille):
        Tg=liste_t[i]
        T_e=methode_explicite(profondeur,N,Tg,Td,T_initial,lambda_conductivite_thermique_sol,capacite_thermique_volumique_sol,delta_t,n)
        T_initial=T_e
        T_courbes.append(T_e)
        
    return T_courbes


        
def evolution_jour_e(liste_t,profondeur,N,Td,lambda_conductivite_thermique_sol,capacite_thermique_volumique_sol):
    taille=len(liste_t)
    
    T_initial=initialisation_zeros(N)
    delta_t=1
    n=3600
    
    T_courbes=[]
    
    for i in range(taille):
        Tg=liste_t[i]
        T_e=methode_explicite(profondeur,N,Tg,Td,T_initial,lambda_conductivite_thermique_sol,capacite_thermique_volumique_sol,delta_t,n)
        T_initial=T_e
        T_courbes.append(T_e)
        
    return T_courbes



In [11]:
# Méthode implicite

def evolution_mois_i(liste_t,profondeur,N,Td,lambda_conductivite_thermique_sol,capacite_thermique_volumique_sol):
    taille=len(liste_t)
    
    T_initial=initialisation_zeros(N)
    delta_t=1
    n=100000
    
    T_courbes=[]
    
    for i in range(taille):
        Tg=liste_t[i]
        T_e=methode_implicite(profondeur,N,Tg,Td,T_initial,lambda_conductivite_thermique_sol,capacite_thermique_volumique_sol,delta_t,n)
        T_initial=T_e
        T_courbes.append(T_e)
        
    return T_courbes


def evolution_annee_i(liste_t,profondeur,N,Td,lambda_conductivite_thermique_sol,capacite_thermique_volumique_sol):
    taille=len(liste_t)
    
    T_initial=initialisation_zeros(N)
    delta_t=1
    n=100000
    
    T_courbes=[]
    
    for i in range(taille):
        Tg=liste_t[i]
        T_e=methode_implicite(profondeur,N,Tg,Td,T_initial,lambda_conductivite_thermique_sol,capacite_thermique_volumique_sol,delta_t,n)
        T_initial=T_e
        T_courbes.append(T_e)
    return T_courbes


def evolution_jour_i(liste_t,profondeur,N,Td,lambda_conductivite_thermique_sol,capacite_thermique_volumique_sol):
    taille=len(liste_t)
    
    T_initial=initialisation_zeros(N)
    delta_t=1
    n=3600
    
    T_courbes=[]
    
    for i in range(taille):
        Tg=liste_t[i]
        T_e=methode_implicite(profondeur,N,Tg,Td,T_initial,lambda_conductivite_thermique_sol,capacite_thermique_volumique_sol,delta_t,n)
        T_initial=T_e
        T_courbes.append(T_e)
        
    return T_courbes

In [15]:
# METHODE EXPLICITE

liste_t=temp_months["France"]
x=intervalle_discret(profondeur,N)
T=evolution_mois_e(liste_t,profondeur,N,Td,lambda_conductivite_thermique_sol,capacite_thermique_volumique_sol)

#print(x)
#print(x[1:-1]) # les valeurs x[0] et x[-1] correspondent à Tg et Td
plt.plot(x[1:-1],T[0],label='$Janvier$')
plt.plot(x[1:-1],T[1],label='$Février$')
plt.plot(x[1:-1],T[2],label='$Mars$')
plt.plot(x[1:-1],T[3],label='$Avril$')
plt.plot(x[1:-1],T[4],label='$Mai$')
plt.plot(x[1:-1],T[5],label='$Juin$')
plt.plot(x[1:-1],T[6],label='$Juillet$')
plt.plot(x[1:-1],T[7],label='$Août$')
plt.plot(x[1:-1],T[8],label='$Septembre$')
plt.plot(x[1:-1],T[9],label='$Octobre$')
plt.plot(x[1:-1],T[10],label='$Novembre$')
plt.plot(x[1:-1],T[11],label='$Décembre$')

#fig = plt.figure(figsize=(10, 5))
plt.xlabel('x la profondeur', fontsize=15)
plt.xlabel('T la température', fontsize=15)
plt.title('Les températures des couches du sol en fonction de la profondeur et du mois correspondant', fontsize=20)
plt.legend(loc='upper right')
plt.show()

NameError: name 'temp_months' is not defined

In [13]:
# TEST METHODE IMPLICITE

x=intervalle_discret(profondeur,N)
T=evolution_mois_i(liste_t,profondeur,N,Td,lambda_conductivite_thermique_sol,capacite_thermique_volumique_sol)

plt.plot(x,T[0],label='$Janvier$')
plt.plot(x,T[1],label='$Février$')
plt.plot(x,T[2],label='$Mars$')
plt.plot(x,T[3],label='$Avril$')
plt.plot(x,T[4],label='$Mai$')
plt.plot(x,T[5],label='$Juin$')
plt.plot(x,T[6],label='$Juillet$')
plt.plot(x,T[7],label='$Août$')
plt.plot(x,T[8],label='$Septembre$')
plt.plot(x,T[9],label='$Octobre')
plt.plot(x,T[10],label='$Novembre$')
plt.plot(x,T[11],label='$Décembre$')

plt.xlabel('x la profondeur', fontsize=15)
plt.xlabel('T la température', fontsize=15)
plt.title('Les températures des couches du sol en fonction de la profondeur et du mois correspondant', fontsize=20)
plt.legend(loc='upper right')
plt.show()

NameError: name 'liste_t' is not defined

In [14]:
#CASE TESTS RAPPEL LISTES
# On teste une liste de listes pour l'affichage
T=[]
T1=[1,2]
T2=[3,4]
T.append(T1)
T.append(T2)

print(T)

[[1, 2], [3, 4]]
